In [1]:
import torch
from utils.loading_utils import load_model, get_device
import numpy as np
import argparse
import pandas as pd
from utils.event_readers import FixedSizeEventReader, FixedDurationEventReader
from utils.inference_utils import events_to_voxel_grid, events_to_voxel_grid_pytorch
from utils.timers import Timer
import time 
from image_reconstructor import ImageReconstructor
from options.inference_options import set_inference_options


In [8]:
parser = argparse.ArgumentParser(
        description='Evaluating a trained network')
parser.add_argument('-c', '--path_to_model', required=True, type=str,
                    help='path to model weights')
parser.add_argument('-i', '--input_file', required=True, type=str)
# parser.add_argument('--fixed_duration', dest='fixed_duration', action='store_true')
# parser.set_defaults(fixed_duration=False)
# parser.add_argument('-N', '--window_size', default=None, type=int,
#                     help="Size of each event window, in number of events. Ignored if --fixed_duration=True")
# parser.add_argument('-T', '--window_duration', default=33.33, type=float,
#                     help="Duration of each event window, in milliseconds. Ignored if --fixed_duration=False")
# parser.add_argument('--num_events_per_pixel', default=0.35, type=float,
#                     help='in case N (window size) is not specified, it will be \
#                             automatically computed as N = width * height * num_events_per_pixel')
# parser.add_argument('--skipevents', default=0, type=int)
# parser.add_argument('--suboffset', default=0, type=int)
# parser.add_argument('--compute_voxel_grid_on_cpu', dest='compute_voxel_grid_on_cpu', action='store_true')
# parser.set_defaults(compute_voxel_grid_on_cpu=False)

# set_inference_options(parser)

args = parser.parse_args()



usage: ipykernel_launcher.py [-h] -c PATH_TO_MODEL -i INPUT_FILE
ipykernel_launcher.py: error: the following arguments are required: -c/--path_to_model, -i/--input_file


SystemExit: 2

In [11]:
path_to_model = "pretrained/E2VID_lightweight.pth.tar"
input_file = "data/dynamic_6dof.zip"


In [26]:
pd.read_csv(input_file, delim_whitespace=True, header=None,
                                    names=['t', 'x', 'y', 'pol'],
                                    dtype={'t': np.float64, 'x': np.int16, 'y': np.int16, 'pol': np.int16},
                                    engine='c',
                                    skiprows= 1, chunksize=10000, nrows=None, memory_map=True)

In [46]:
reader = pd.read_csv(input_file, delim_whitespace=True, header=None,
                                    names=['t', 'x', 'y', 'pol'],
                                    nrows=2)

In [47]:
print(reader)

              t    x     y  pol
0  2.400000e+02  180   NaN  NaN
1  1.473348e+09   80  22.0  0.0


In [12]:
# Read sensor size from the first first line of the event file
path_to_events = input_file

header = pd.read_csv(path_to_events, delim_whitespace=True, header=None, names=['width', 'height'],
                        dtype={'width': int, 'height': int},
                        nrows=1)
    
width, height = header.values[0]
print('Sensor size: {} x {}'.format(width, height))

Sensor size: 240 x 180


In [15]:
header.values[0]

array([240, 180])

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = load_model(path_to_model, device)
model = model.to(device)
    

Loading model pretrained/E2VID_lightweight.pth.tar...
Using TransposedConvLayer (fast, with checkerboard artefacts)


In [18]:
model.eval()

E2VIDRecurrent(
  (unetrecurrent): UNetRecurrent(
    (head): ConvLayer(
      (conv2d): Conv2d(5, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
    (encoders): ModuleList(
      (0): RecurrentConvLayer(
        (conv): ConvLayer(
          (conv2d): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
          (norm_layer): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (recurrent_block): ConvLSTM(
          (Gates): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (1): RecurrentConvLayer(
        (conv): ConvLayer(
          (conv2d): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
          (norm_layer): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (recurrent_block): ConvLSTM(
          (Gates): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    

In [19]:
model.num_bins

5

In [20]:
model.num_encoders

3

In [3]:
import torch
ts = 3.7
pols = +1
ts = torch.tensor(ts)
pols = torch.tensor(pols)

tis = torch.floor(ts)  #向下取整
tis_long = tis.long()  #转为Int64
dts = ts - tis         #找到小数
vals_left = pols * (1.0 - dts.float())
vals_right = pols * dts.float()

print(tis,
tis_long,
dts,
vals_left,
vals_right)

tensor(3.) tensor(3) tensor(0.7000) tensor(0.3000) tensor(0.7000)


In [8]:
valid_indices = tis < 5
print(valid_indices,valid_indices.shape)

tensor(True) torch.Size([])


In [16]:

valid_indices &= tis >= 100
valid_indices
a=True
b=True
a &= b
print(a)


True


In [4]:
import torch
a=torch.rand(2,4)
print(a)
print(a[:,:1])
print(a[:,1:])

tensor([[0.7254, 0.1629, 0.4855, 0.7100],
        [0.5247, 0.0156, 0.5747, 0.6553]])
tensor([[0.7254],
        [0.5247]])
tensor([[0.1629, 0.4855, 0.7100],
        [0.0156, 0.5747, 0.6553]])


In [8]:
import torch
a=torch.randn(4,1) #随机生成一个shape（3，4）的tensort
b=torch.randn(4,3) #随机生成一个shape（2，4）的tensor

c = torch.cat([a,b],dim=1) 
print(c.shape)

torch.Size([4, 4])


In [7]:
import torch
a = torch.rand(10, requires_grad=True).cuda()
b = a+1

b.sum().backward()

In [8]:
print(b.grad)
print(a.grad)

None
None


/home/minghong@corp.sse.tongji.edu.cn/.conda/envs/E2VID/lib/python3.9/site-packages/torch/_tensor.py:1104: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/build/aten/src/ATen/core/TensorBody.h:475.)
  return self._grad


In [3]:
import torch

a1 = torch.rand([4, 4], requires_grad=True).squeeze(0)
b1 = a1**2
b1.sum().backward()
print(a1.grad)
print(b1.grad)
a2 = torch.rand([1, 4, 4], requires_grad=True).unsqueeze(0)
b2 = a2**2
b2.sum().backward()
print(a2.grad)
print(b2.grad)

None
None
None
None


/home/minghong@corp.sse.tongji.edu.cn/.conda/envs/E2VID/lib/python3.9/site-packages/torch/_tensor.py:1104: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/build/aten/src/ATen/core/TensorBody.h:475.)
  return self._grad


In [14]:
import torch

a1 = torch.rand([4, 4], requires_grad=True)

c1 = a1 * 2 
b1 = c1.squeeze(0)**2
a1.retain_grad()
b1.sum().backward()
print(a1.is_leaf)
print(a1.grad.shape)
print(a1.shape)
a2 = torch.rand([1, 4, 4], requires_grad=True)
b2 = a2.unsqueeze(0)**2
b2.sum().backward()
print(a2.is_leaf)
print(a2.grad)
print(a2.grad.shape)
print(a2.shape)


True
torch.Size([4, 4])
torch.Size([4, 4])
True
tensor([[[0.6153, 1.5961, 0.4122, 0.5027],
         [1.8332, 1.1463, 1.7687, 0.2403],
         [0.2135, 1.7001, 0.5735, 1.6498],
         [0.1901, 1.9154, 1.5768, 0.9390]]])
torch.Size([1, 4, 4])
torch.Size([1, 4, 4])


In [11]:
import torch

a1 = torch.rand([4, 4], requires_grad=True)

a1 = a1 * 2 
b1 = a1.squeeze(0)**2
a1.retain_grad()
b1.sum().backward()
print(a1.is_leaf)
print(a1.grad)

False
tensor([[1.3048, 0.3445, 2.9272, 3.9529],
        [1.7340, 3.0695, 3.4180, 1.3781],
        [3.8073, 1.4407, 1.6093, 1.3925],
        [0.2178, 3.6619, 0.6721, 2.2217]])


In [16]:
import torch
a = torch.Tensor([1,2]).requires_grad_() 
b = torch.Tensor([3,4]).requires_grad_() 
d = torch.Tensor([2]).requires_grad_() 
c = a + b 
e = c * d 
o = e.sum()     
 
o.backward()
print('a: ',a.is_leaf,' ',a.grad)
print('b: ',b.is_leaf,' ',b.grad)
print('c: ',c.is_leaf,' ',c.grad)
print('d: ',d.is_leaf,' ',d.grad)
print('e: ',e.is_leaf,' ',e.grad)
print('o: ',o.is_leaf,' ',o.grad)

a:  True   tensor([2., 2.])
b:  True   tensor([2., 2.])
c:  False   None
d:  True   tensor([10.])
e:  False   None
o:  False   None


/home/minghong@corp.sse.tongji.edu.cn/.conda/envs/E2VID/lib/python3.9/site-packages/torch/_tensor.py:1104: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/build/aten/src/ATen/core/TensorBody.h:475.)
  return self._grad


In [20]:
import torch
a = torch.Tensor([1,2]).requires_grad_() 
b = torch.Tensor([3,4]).requires_grad_() 
d = torch.Tensor([2]).requires_grad_() 
c = a + b 
e = c * d 
o = e.sum()     
 
o.backward()
print('a: ',a.is_leaf,' ',a.grad)
print('b: ',b.is_leaf,' ',b.grad)
print('c: ',c.is_leaf,' ',c.grad)
print('d: ',d.is_leaf,' ',d.grad)
print('e: ',e.is_leaf,' ',e.grad)
print('o: ',o.is_leaf,' ',o.grad)

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

a = [3,2,1]

In [25]:
import torch
a = torch.Tensor([1.233,2.232])
print(a.dtype)
a = a.long()
print(a.dtype)
print(a)
b = torch.Tensor([1473347517.019522666931])
b = b* (10**10)

torch.float32
torch.int64
tensor([1, 2])
